In this notebook, I will use the information about IA to refine F3. Initially, I buildt F3 using FEF25-75%FEV1 against FEV1-based-AR. The latter variable contains only the aspect of AR as measured by FEV1. In CF, high AR usually correlates with high IA. We can use this correlation to refine the uncertainty present in the FEV1-based-AR. The corrected FEV1-based-AR can therefore be closer to the true AR. We can use it to improve the model fo F3.

In [1]:
import src.data.breathe_data as br
import src.data.helpers as dh
import src.models.helpers as mh
import numpy as np
import pandas as pd
import src.inference.helpers as ih
import src.modelling_fef2575.cpt_and_plots as cpt_and_plots

In [2]:
# Need dataset with O2sat, FEV1, FEF25-75. Use as many datapoints as possible
# Infer FEV1-based-AR using FEV1
# Model F3 using FEV1-based-AR
# Infer FEV1-FEF2575-based-AR using FEV1 and FEF25-75
# Model F3 using FEV1-FEF2575-based-AR
# Infer IA using FEV1-FEF2575-based-AR
# Model AR-IA
# Infer IA-FEV1-FEF2575-based-AR using FEV1, FEF25-75, IA
# Model F3 using this new AR
# Compare the two models: compare the mean, median, std-percentiles plots of both -> std should be smaller

# Optionally repeat until std doesn't change

In [3]:
df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan")
# Remove PEF related rows
df = df.drop(columns=["PEF", "ecPEF (L/s)", "PEF (L/s)"])
# Remove NaN on FEV1, O2 saturation columns
print(df.shape)
df = df.dropna(subset=["FEV1", "O2 Saturation", "FEF2575"])
print(df.shape)
df.head()

(48978, 15)
(32715, 15)


,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
0,101,2019-01-25,1.31,97.0,0.54,1.31,0.67,53,Male,173.0,3.610061,97.150104,36.287474,36.287474,99.845492
1,101,2019-01-26,1.31,98.0,0.57,1.31,0.67,53,Male,173.0,3.610061,97.150104,36.287474,36.287474,100.874827
2,101,2019-01-27,1.31,96.0,0.67,1.31,0.69,53,Male,173.0,3.610061,97.150104,36.287474,36.287474,98.816157
3,101,2019-01-28,1.30,96.0,0.69,1.31,0.69,53,Male,173.0,3.610061,97.150104,36.287474,36.010470,98.816157
4,101,2019-01-29,1.28,98.0,0.60,1.30,0.69,53,Male,173.0,3.610061,97.150104,36.010470,35.456463,100.874827


In [4]:
# Infer FEV1-based-AR using FEV1

# inf_res_df = ih.infer_AR_IA_HFEV1_HO2sat_get_back_df(df,  observed_variables=["ecFEV1", "O2Sat"])
inf_res_df = ih.infer_AR_IA_HFEV1_HO2sat_get_back_df(
    df.iloc[np.r_[10:1300, 3000:4007]],
    ecFEF2575prctecFEV1_cpt=None,
    observed_variables=["ecFEV1", "O2Sat"],
)
# inf_res_df.head()

In [5]:
# Merge the inferred AR back to the original dataframe
df1 = pd.merge(df, inf_res_df, on=["ID", "Date Recorded"], how="inner")
# df1.head()

In [6]:
# Model F3 using FEV1-based-AR

def model_f3(df, ar_col):
    df["ecFEF2575%ecFEV1"] = df["ecFEF2575"] / df["ecFEV1"] * 100

    # Parameters
    n_samples = 100

    ecFEF2575prctecFEV1 = mh.VariableNode("ecFEF25-75 % ecFEV1 (%)", 0, 200, 2, prior=None)
    AR = mh.VariableNode("Airway resistance (%)", 0, 90, 2, prior=None)

    df_sampled, df_f3 = cpt_and_plots.get_sampled_df_and_statistics_df(df, n_samples, AR)

    cpt_and_plots.plot_F3_mean_and_percentiles_per_AR_bin(
        df_f3, f"F3 - ecFEF25-75%ecFEV1 statistics per {ar_col} bin", save=True
    )
    cpt_f3 = cpt_and_plots.calc_and_plot_FEF2575prctFEV1_AR_cpt(
        df_sampled, df_f3, n_samples, AR, ecFEF2575prctecFEV1, ar_col, save=True
    )
    return cpt_f3

cpt_f3 = model_f3(df1, "FEV1-based-AR")

Max sampled AR values: 81.29


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/modelling_fef2575/cpt_and_plots.py:234: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_sampled.groupby("AR bin")


In [7]:
inf_res_df_2 = ih.infer_AR_IA_HFEV1_HO2sat_get_back_df(
    df.iloc[np.r_[10:1300, 3000:4007]],
    ecFEF2575prctecFEV1_cpt=cpt_f3,
    observed_variables=["ecFEV1", "O2Sat", "ecFEF2575prctecFEV1"],
)
# Merge the inferred AR back to the original dataframe
df2 = pd.merge(df, inf_res_df_2, on=["ID", "Date Recorded"], how="inner")

In [8]:
# Model F3 using FEV1-FEF2575-based-AR
cpt_f3_2 = model_f3(df2, "FEV1-FEF2575-based-AR")

Max sampled AR values: 81.67


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/modelling_fef2575/cpt_and_plots.py:234: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:1983: RuntimeWarning:

divide by zero encountered in divide

